In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Week 1: Sentiment in text

## Tokenizer

In [2]:
scentences = ["I love my dog", "I love my cat", "you love my dog!"]

# <OOV>: <UNK>
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(scentences)

tokenizer.word_index

{'<OOV>': 1, 'love': 2, 'my': 3, 'i': 4, 'dog': 5, 'cat': 6, 'you': 7}

## Text to Sequence

In [3]:
scentences.append("love love love my dog dog dog he is amazing")

In [4]:
sequences = tokenizer.texts_to_sequences(scentences)
sequences

[[4, 2, 3, 5], [4, 2, 3, 6], [7, 2, 3, 5], [2, 2, 2, 3, 5, 5, 5, 1, 1, 1]]

In [5]:
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding = "post", maxlen=7, truncating="pre")
padded

array([[4, 2, 3, 5, 0, 0, 0],
       [4, 2, 3, 6, 0, 0, 0],
       [7, 2, 3, 5, 0, 0, 0],
       [3, 5, 5, 5, 1, 1, 1]], dtype=int32)

## Sarcasm Dataset

In [16]:
# Download the dataset
# !wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
import json

# Load the JSON file
with open("./sarcasm.json", 'r') as f:
    datastore = json.load(f)
# Initialize lists
sentences = [] 
labels = []
urls = []

# Append elements in the dictionaries into each list
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [20]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)

print(len(tokenizer.word_index))

29657


In [23]:
sequences = tokenizer.texts_to_sequences(sentences)
print(len(sequences))
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding = "post")

26709


In [24]:
print(padded[0])
print(padded.shape)

[  308 15115   679  3337  2298    48   382  2576 15116     6  2577  8434
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
(26709, 40)


# Week 2: Word Embeddings

## Downloading the dataset using `tensorflow_datasets`

In [10]:
import tensorflow_datasets as tfds
# imdb, info = tfds.load("imdb_reviews", with_info= True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQELMI0/imdb_reviews-train.tfrecord*...…

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQELMI0/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQELMI0/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2022-09-20 19:53:16.725622: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-20 19:53:16.810432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-20 19:53:16.810991: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-20 19:53:16.812960: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [17]:
print(imdb)

{Split('train'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('test'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('unsupervised'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [30]:
print(training_sentences[0])
print(training_labels[0:5])

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
[0, 0, 0, 1, 1]


In [46]:
training_data, testing_data = imdb["train"], imdb["test"]

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in training_data:
    training_sentences.append(s.numpy().decode("utf8"))
    training_labels.append(l.numpy())

training_labels = np.array(training_labels)

for s, l in testing_data:
    testing_sentences.append(s.numpy().decode("utf8"))
    testing_labels.append(l.numpy())

testing_labels = np.array(testing_labels)


In [53]:
training_labels.shape

(25000,)

## Tokenizer and padded sequences

In [105]:
1e1

10.0

In [106]:
# Tokenization parameters
vocab_size = int(1e4)
max_length = 120
# Embedding vector length
embedding_dim = 16
trunc_type = "post"
oov_token = "<OOV>"

In [107]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token = oov_token)

tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = tf.keras.preprocessing.sequence.pad_sequences(testing_sequences, maxlen = max_length, truncating = trunc_type)

In [113]:
print(training_sentences[0])
print(padded[0])
print(training_labels[0:5])

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
[   0    0    0   12   14   33  425  392   18   90   28    1    9   32
 1366 3585   40  486    1  197   24   85  154   19   12  213  329   28
   66  247  215    9  477   58   66   85  114   98   22 5675   12 1322
  643  767   12   18    7   33  400 8170  176 2455  416    2   89 1231
  137 

## Building The Model

In [109]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d_5   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_10 (Dense)            (None, 6)                 102       
                                                                 
 dense_11 (Dense)            (None, 1)                 7         
                                                                 
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [111]:
model.fit(
    padded,
    training_labels,
    epochs = 10,
    validation_data = (testing_padded, testing_labels)
)

Epoch 1/10
782/782 [==============================] - 11s 14ms/step - loss: 0.1251 - accuracy: 0.9600 - val_loss: 0.6014 - val_accuracy: 0.8075
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.1151 - accuracy: 0.9636 - val_loss: 0.6429 - val_accuracy: 0.8009
Epoch 3/10
782/782 [==============================] - 9s 12ms/step - loss: 0.1050 - accuracy: 0.9678 - val_loss: 0.6903 - val_accuracy: 0.7964
Epoch 4/10
782/782 [==============================] - 11s 14ms/step - loss: 0.0953 - accuracy: 0.9732 - val_loss: 0.7466 - val_accuracy: 0.7986
Epoch 5/10
782/782 [==============================] - 12s 16ms/step - loss: 0.0854 - accuracy: 0.9767 - val_loss: 0.8094 - val_accuracy: 0.7949
Epoch 6/10
782/782 [==============================] - 12s 15ms/step - loss: 0.0780 - accuracy: 0.9795 - val_loss: 0.8786 - val_accuracy: 0.7928
Epoch 7/10
782/782 [==============================] - 11s 14ms/step - loss: 0.0705 - accuracy: 0.9827 - val_loss: 0.8951 - val_accuracy: 

## saving weights for visualization

In [ ]:
embedding_weights = model.layers[0].get_weights()[0]
embedding_weights.shape

(100000, 16)

In [115]:
import io
out_v = io.open("vecs.tsv", "w", encoding= "utf-8")
out_m = io.open("meta.tsv", "w", encoding= "utf-8")

for word_num in range(1, vocab_size):
    word_embedding = embedding_weights[word_num]
    word_name = tokenizer.index_word[word_num]

    out_m.write(word_name + "\n")
    out_v.write("\t".join([str(x) for x in word_embedding]) + "\n")

out_v.close()
out_m.close()


## Sub words tokenization

`Ten sor Fl ow from basic cs to master`
- case sensitive
- punctuation remains

➡️ better results

# Week 3: Sequence Models

## LSTM / GRU Unit

In [ ]:
inputs2 = tf.keras.layers.Input(shape = (3,1))
# return_sequences = True ➡️ 3 hidden states
lstm2 = tf.keras.layers.LSTM(1, return_sequences = True)(inputs2)
model = tf.keras.models.Model(inputs2, lstm2)

model.predict(np.array([0.1,0.2,0.3]).reshape(1,3,1))

## The model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    # return_seuqnces: output of the first matches the input of the second
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

## Notes:
- Using different kinds of layers often causes overfitting (LSTM vs NonLSTM Example)
- Conv1D can be used, and it's faster
- overfitting often happens in text because some <UNK> words

# Week 4: Generating Text

## Preparing the training data

In [7]:
# data = "In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."

In [92]:
data = "Mr. and Mrs. Dursley, of number four, Privet Drive, were \n proud to say that they were perfectly normal, thank \n you very much. ' ey were the last people you'd expect to be in'volved in anything strange or mysterious, because they just didn't \n hold with such nonsense.\n Mr. Dursley was the director of a fi rm called Grunnings, which \n made drills. He was a big, beefy man with hardly any neck, al'though he did have a very large mustache. Mrs. Dursley was thin \n and blonde and had nearly twice the usual amount of neck, which \n came in very useful as she spent so much of her time craning over \n garden fences, spying on the neighbors. ' e Dursleys had a small \n son called Dudley and in their opinion there was no fi ner boy \n anywhere.\n ' e Dursleys had everything they wanted, but they also had a \n secret, and their greatest fear was that somebody would discover it. \n ' ey didn't think they could bear it if anyone found out about the \n Potters. Mrs. Potter was Mrs. Dursley's sister, but they hadn't met \n for several years; in fact, Mrs. Dursley pretended she didn't have a \n sister, because her sister and her good-for-nothing husband were \n as unDursleyish as it was possible to be. ' e Dursleys shuddered \n to think what the neighbors would say if the Potters arrived in the \n street. ' e Dursleys knew that the Potters had a small son, too, but \n they had never even seen him. ' is boy was another good reason \n for keeping the Potters away; they didn't want Dudley mixing with \n a child like that.\n When Mr. and Mrs. Dursley woke up on the dull, gray Tuesday \n our story starts, there was nothing about the cloudy sky outside to \n suggest that strange and mysterious things would soon be happen'ing all over the country. Mr. Dursley hummed as he picked out his \n most boring tie for work, and Mrs. Dursley gossiped away happily \n as she wrestled a screaming Dudley into his high chair.\n None of them noticed a large, tawny owl fl utter past the \n window.\n At half past eight, Mr. Dursley picked up his briefcase, pecked \n Mrs. Dursley on the cheek, and tried to kiss Dudley good-bye but \n missed, because Dudley was now having a tantrum and throwing \n his cereal at the walls. “Little tyke,'' chortled Mr. Dursley as he left \n the house. He got into his car and backed out of number four's \n drive.\n It was on the corner of the street that he noticed the fi rst sign \n of something peculiar — a cat reading a map. For a second, Mr. \n Dursley didn't realize what he had seen — then he jerked his head \n around to look again. ' ere was a tabby cat standing on the corner \n of Privet Drive, but there wasn't a map in sight. What could he \n have been thinking of? It must have been a trick of the light. Mr. \n Dursley blinked and stared at the cat. It stared back. As Mr. Durs'ley drove around the corner and up the road, he watched the cat in \n his mirror. It was now reading the sign that said Privet Drive — no, \n looking at the sign; cats couldn't read maps or signs. Mr. Dursley \n gave himself a little shake and put the cat out of his mind. As he \n drove toward town he thought of nothing except a large order of \n drills he was hoping to get that day.\n But on the edge of town, drills were driven out of his mind \n by something else. As he sat in the usual morning traffi c jam, he \n couldn't help noticing that there seemed to be a lot of strangely \n dressed people about. People in cloaks. Mr. Dursley couldn't bear \n people who dressed in funny clothes — the getups you saw on \n young people! He supposed this was some stupid new fashion. He \n drummed his fi ngers on the steering wheel and his eyes fell on a \n huddle of these weirdos standing quite close by. ' ey were whis'pering excitedly together. Mr. Dursley was enraged to see that a \n couple of them weren't young at all; why, that man had to be older \n than he was, and wearing an emerald-green cloak! ' e nerve of \n him! But then it struck Mr. Dursley that this was probably some \n silly stunt — these people were obviously collecting for something \n . . . yes, that would be it. ' e traffi c moved on and a few minutes \n later, Mr. Dursley arrived in the Grunnings parking lot, his mind \n back on drills.\n Mr. Dursley always sat with his back to the window in his offi ce \n on the ninth fl oor. If he hadn't, he might have found it harder to \n concentrate on drills that morning. He didn't see the owls swooping \n past in broad daylight, though people down in the street did; they \n pointed and gazed open-mouthed as owl after owl sped overhead. \n Most of them had never seen an owl even at nighttime. Mr. Durs'ley, however, had a perfectly normal, owl-free morning. He yelled \n at fi ve diff erent people. He made several important telephone calls \n and shouted a bit more. He was in a very good mood until lunch'time, when he thought he'd stretch his legs and walk across the \n road to buy himself a bun from the bakery.\n He'd forgotten all about the people in cloaks until he passed \n a group of them next to the baker's. He eyed them angrily as he \n passed. He didn't know why, but they made him uneasy. ' is \n bunch were whispering excitedly, too, and he couldn't see a single \n collecting tin. It was on his way back past them, clutching a large \n doughnut in a bag, that he caught a few words of what they were \n saying.\n “' e Potters, that's right, that's what I heard —''\n “— yes, their son, Harry —''\n Mr. Dursley stopped dead. Fear fl ooded him. He looked back \n at the whisperers as if he wanted to say something to them, but \n thought better of it.\n He dashed back across the road, hurried up to his offi ce, snapped \n at his secretary not to disturb him, seized his telephone, and had \n almost fi nished dialing his home number when he changed his \n mind. He put the receiver back down and stroked his mustache, \n thinking . . . no, he was being stupid. Potter wasn't such an un'usual name. He was sure there were lots of people called Potter who \n had a son called Harry. Come to think of it, he wasn't even sure his \n nephew was called Harry. He'd never even seen the boy. It might \n have been Harvey. Or Harold. ' ere was no point in worrying \n Mrs. Dursley; she always got so upset at any mention of her sister. \n He didn't blame her — if he'd had a sister like that . . . but all the \n same, those people in cloaks . . . \n He found it a lot harder to concentrate on drills that afternoon \n and when he left the building at fi ve o'clock, he was still so worried \n that he walked straight into someone just outside the door.\n “Sorry,'' he grunted, as the tiny old man stumbled and almost \n fell. It was a few seconds before Mr. Dursley realized that the man \n was wearing a violet cloak. He didn't seem at all upset at being al'most knocked to the ground. On the contrary, his face split into a \n wide smile and he said in a squeaky voice that made passersby stare, \n “Don't be sorry, my dear sir, for nothing could upset me today! Re'joice, for You-Know-Who has gone at last! Even Muggles like your'self should be celebrating, this happy, happy day!''\n And the old man hugged Mr. Dursley around the middle and \n walked off .\n Mr. Dursley stood rooted to the spot. He had been hugged by a \n complete stranger. He also thought he had been called a Muggle, \n whatever that was. He was rattled. He hurried to his car and set \n off for home, hoping he was imagining things, which he had never \n hoped before, because he didn't approve of imagination.\n As he pulled into the driveway of number four, the fi rst thing he \n saw — and it didn't improve his mood — was the tabby cat he'd \n spotted that morning. It was now sitting on his garden wall. He \n was sure it was the same one; it had the same markings around its \n eyes.\n “Shoo!'' said Mr. Dursley loudly.\n ' e cat didn't move. It just gave him a stern look. Was this nor'mal cat behavior? Mr. Dursley wondered. Trying to pull himself to'gether, he let himself into the house. He was still determined not to \n mention anything to his wife.\n Mrs. Dursley had had a nice, normal day. She told him over din'ner all about Mrs. Next Door's problems with her daughter and how \n Dudley had learned a new word (“Won't!''). Mr. Dursley tried to \n act normally. When Dudley had been put to bed, he went into the \n living room in time to catch the last report on the evening news:\n “And fi nally, bird-watchers everywhere have reported that the \n nation's owls have been behaving very unusually today. Although \n owls normally hunt at night and are hardly ever seen in daylight, \n there have been hundreds of sightings of these birds fl ying in every \n direction since sunrise. Experts are unable to explain why the owls \n have suddenly changed their sleeping pattern.'' ' e newscaster \n allowed himself a grin. “Most mysterious. And now, over to Jim \n McGuffi n with the weather. Going to be any more showers of owls \n tonight, Jim?''\n “Well, Ted,'' said the weatherman, “I don't know about that, but \n it's not only the owls that have been acting oddly today. Viewers as \n far apart as Kent, Yorkshire, and Dundee have been phoning in to \n tell me that instead of the rain I promised yesterday, they've had a \n downpour of shooting stars! Perhaps people have been celebrating \n Bonfi re Night early — it's not until next week, folks! But I can \n promise a wet night tonight.''\n Mr. Dursley sat frozen in his armchair. Shooting stars all over \n Britain? Owls fl ying by daylight? Mysterious people in cloaks all \n over the place? And a whisper, a whisper about the Potters . . . \n Mrs. Dursley came into the living room carrying two cups of \n tea. It was no good. He'd have to say something to her. He cleared \n his throat nervously. “Er — Petunia, dear — you haven't heard \n from your sister lately, have you?''\n As he had expected, Mrs. Dursley looked shocked and angry. Af'ter all, they normally pretended she didn't have a sister.\n “No,'' she said sharply. “Why?''\n “Funny stuff on the news,'' Mr. Dursley mumbled. “Owls . . . \n shooting stars . . . and there were a lot of funny-looking people in \n town today . . .''\n “So?'' snapped Mrs. Dursley.\n “Well, I just thought . . . maybe . . . it was something to do \n with . . . you know . . . her crowd.''\n Mrs. Dursley sipped her tea through pursed lips. Mr. Dursley \n wondered whether he dared tell her he'd heard the name “Potter.'' \n He decided he didn't dare. Instead he said, as casually as he could, \n “' eir son — he'd be about Dudley's age now, wouldn't he?''\n “I suppose so,'' said Mrs. Dursley stiffl y.\n “What's his name again? Howard, isn't it?''\n “Harry. Nasty, common name, if you ask me.''\n “Oh, yes,'' said Mr. Dursley, his heart sinking horribly. “Yes, I \n quite agree.''\n He didn't say another word on the subject as they went upstairs \n to bed. While Mrs. Dursley was in the bathroom, Mr. Dursley \n crept to the bedroom window and peered down into the front gar'den. ' e cat was still there. It was staring down Privet Drive as \n though it were waiting for something.\n Was he imagining things? Could all this have anything to do \n with the Potters? If it did . . . if it got out that they were related to \n a pair of — well, he didn't think he could bear it.\n ' e Dursleys got into bed. Mrs. Dursley fell asleep quickly but \n Mr. Dursley lay awake, turning it all over in his mind. His last, \n comforting thought before he fell asleep was that even if the Potters \n were involved, there was no reason for them to come near him and \n Mrs. Dursley. ' e Potters knew very well what he and Petunia \n thought about them and their kind. . . . He couldn't see how he \n and Petunia could get mixed up in anything that might be going \n on — he yawned and turned over — it couldn't aff ect them. . . . \n How very wrong he was.\n Mr. Dursley might have been drifting into an uneasy sleep, but \n the cat on the wall outside was showing no sign of sleepiness. It was \n sitting as still as a statue, its eyes fi xed unblinkingly on the far cor'ner of Privet Drive. It didn't so much as quiver when a car door \n slammed on the next street, nor when two owls swooped overhead. \n In fact, it was nearly midnight before the cat moved at all.\n A man appeared on the corner the cat had been watching, ap'peared so suddenly and silently you'd have thought he'd just \n popped out of the ground. ' e cat's tail twitched and its eyes nar'rowed.\n Nothing like this man had ever been seen on Privet Drive. He \n was tall, thin, and very old, judging by the silver of his hair and \n beard, which were both long enough to tuck into his belt. He was \n wearing long robes, a purple cloak that swept the ground, and \n high-heeled, buckled boots. His blue eyes were light, bright, and \n sparkling behind half-moon spectacles and his nose was very long \n and crooked, as though it had been broken at least twice. ' is \n man's name was Albus Dum ble dore.\n Albus Dum ble dore didn't seem to realize that he had just arrived \n in a street where everything from his name to his boots was unwel'come. He was busy rummaging in his cloak, looking for some'thing. But he did seem to realize he was being watched, because he \n looked up suddenly at the cat, which was still staring at him from \n the other end of the street. For some reason, the sight of the cat \n seemed to amuse him. He chuckled and muttered, “I should have \n known.''\n He found what he was looking for in his inside pocket. It \n seemed to be a silver cigarette lighter. He fl icked it open, held it up \n in the air, and clicked it. ' e nearest street lamp went out with a \n little pop. He clicked it again — the next lamp fl ickered into dark'ness. Twelve times he clicked the Put-Outer, until the only lights \n left on the whole street were two tiny pinpricks in the distance, \n which were the eyes of the cat watching him. If anyone looked out \n of their window now, even beady-eyed Mrs. Dursley, they wouldn't \n be able to see anything that was happening down on the pavement. \n Dum ble dore slipped the Put-Outer back inside his cloak and set \n off down the street toward number four, where he sat down on the \n wall next to the cat. He didn't look at it, but after a moment he \n spoke to it.\n “Fancy seeing you here, Professor McGonagall.''\n He turned to smile at the tabby, but it had gone. Instead he was \n smiling at a rather severe-looking woman who was wearing square \n glasses exactly the shape of the markings the cat had had around its \n eyes. She, too, was wearing a cloak, an emerald one. Her black hair \n was drawn into a tight bun. She looked distinctly ruffl ed.\n “How did you know it was me?'' she asked.\n “My dear Professor, I've never seen a cat sit so stiffl y.''\n “You'd be stiff if you'd been sitting on a brick wall all day,'' said \n Professor McGonagall.\n “All day? When you could have been celebrating? I must have \n passed a dozen feasts and parties on my way here.''\n Professor McGonagall sniff ed angrily.\n “Oh yes, everyone's celebrating, all right,'' she said impatiently. \n “You'd think they'd be a bit more careful, but no — even the Mug'gles have noticed something's going on. It was on their news.'' She \n jerked her head back at the Dursleys' dark living-room window. “I \n heard it. Flocks of owls . . . shooting stars. . . . Well, they're not \n completely stupid. ' ey were bound to notice something. Shoot'ing stars down in Kent — I'll bet that was Dedalus Diggle. He \n never had much sense.''\n “You can't blame them,'' said Dum ble dore gently. “We've had \n precious little to celebrate for eleven years.''\n “I know that,'' said Professor McGonagall irritably. “But that's \n no reason to lose our heads. People are being downright careless, \n out on the streets in broad daylight, not even dressed in Muggle \n clothes, swapping rumors.''\n She threw a sharp, sideways glance at Dum ble dore here, as \n though hoping he was going to tell her something, but he didn't, so \n she went on. “A fi ne thing it would be if, on the very day You'Know-Who seems to have disappeared at last, the Muggles found \n out about us all. I suppose he really has gone, Dum ble dore?''\n “It certainly seems so,'' said Dum ble dore. “We have much to be \n thankful for. Would you care for a lemon drop?''\n “A what?''\n “A lemon drop. ' ey're a kind of Muggle sweet I'm rather \n fond of.''\n “No, thank you,'' said Professor McGonagall coldly, as though \n she didn't think this was the moment for lemon drops. “As I say, \n even if You-Know-Who has gone —''\n “My dear Professor, surely a sensible person like yourself can call \n him by his name? All this 'You-Know-Who' nonsense — for eleven \n years I have been trying to persuade people to call him by his \n proper name: Vol de mort.'' Professor McGonagall fl inched, but Dum'bledore, who was unsticking two lemon drops, seemed not to no'tice. “It all gets so confusing if we keep saying 'You-Know-Who.' I \n have never seen any reason to be frightened of saying Vol de mort's \n name.''\n “I know you haven't,'' said Professor McGonagall, sounding half \n exasperated, half admiring. “But you're diff erent. Everyone knows \n you're the only one You-Know- oh, all right, Vol de mort, was fright'ened of.''\n “You fl atter me,'' said Dum ble dore calmly. “Vol de mort had \n powers I will never have.''\n “Only because you're too — well — noble to use them.''\n “It's lucky it's dark. I haven't blushed so much since Madam \n Pomfrey told me she liked my new earmuff s.''\n Professor McGonagall shot a sharp look at Dum ble dore and \n said, “' e owls are nothing next to the rumors that are fl ying \n around. You know what everyone's saying? About why he's disap'peared? About what fi nally stopped him?''\n It seemed that Professor McGonagall had reached the point she \n was most anxious to discuss, the real reason she had been waiting \n on a cold, hard wall all day, for neither as a cat nor as a woman had \n she fi xed Dum ble dore with such a piercing stare as she did now. It \n was plain that whatever “everyone'' was saying, she was not going \n to believe it until Dum ble dore told her it was true. Dum ble dore, \n however, was choosing another lemon drop and did not answer.\n “What they're saying,'' she pressed on, “is that last night Vol'demort turned up in Godric's Hollow. He went to fi nd the Pot'ters. ' e rumor is that Lily and James Potter are — are — that \n they're — dead.''\n Dum ble dore bowed his head. Professor McGonagall gasped.\n “Lily and James . . . I can't believe it . . . I didn't want to believe \n it . . . Oh, Albus . . .''\n Dum ble dore reached out and patted her on the shoulder. “I \n know . . . I know . . .'' he said heavily.\n Professor McGonagall's voice trembled as she went on. “' at's \n not all. ' ey're saying he tried to kill the Potters' son, Harry. \n But — he couldn't. He couldn't kill that little boy. No one knows \n why, or how, but they're saying that when he couldn't kill Harry \n Potter, Vol de mort's power somehow broke — and that's why he's \n gone.''\n Dum ble dore nodded glumly.\n “It's — it's true?'' faltered Professor McGonagall. “After all he's \n done . . . all the people he's killed . . . he couldn't kill a little boy? \n It's just astounding . . . of all the things to stop him . . . but how in \n the name of heaven did Harry survive?''\n “We can only guess,'' said Dum ble dore. “We may never know.''\n Professor McGonagall pulled out a lace handkerchief and \n dabbed at her eyes beneath her spectacles. Dum ble dore gave a great \n sniff as he took a golden watch from his pocket and examined it. \n It was a very odd watch. It had twelve hands but no numbers; in'stead, little planets were moving around the edge. It must have \n made sense to Dum ble dore, though, because he put it back in his \n pocket and said, “Hagrid's late. I suppose it was he who told you I'd \n be here, by the way?''\n “Yes,'' said Professor McGonagall. “And I don't suppose you're \n going to tell me why you're here, of all places?''\n “I've come to bring Harry to his aunt and uncle. ' ey're the \n only family he has left now.''\n “You don't mean — you can't mean the people who live here?'' \n cried Professor McGonagall, jumping to her feet and pointing at \n number four. “Dum ble dore — you can't. I've been watching them \n all day. You couldn't fi nd two people who are less like us. And \n they've got this son — I saw him kicking his mother all the way up \n the street, screaming for sweets. Harry Potter come and live here!''\n “It's the best place for him,'' said Dum ble dore fi rmly. “His aunt \n and uncle will be able to explain everything to him when he's older. \n I've written them a letter.''\n “A letter?'' repeated Professor McGonagall faintly, sitting back \n down on the wall. “Really, Dum ble dore, you think you can explain \n all this in a letter? ' ese people will never understand him! He'll be \n famous — a legend — I wouldn't be surprised if today was known \n as Harry Potter Day in the future — there will be books written \n about Harry — every child in our world will know his name!''\n “Exactly,'' said Dum ble dore, looking very seriously over the top \n of his half-moon glasses. “It would be enough to turn any boy's \n head. Famous before he can walk and talk! Famous for something \n he won't even remember! Can't you see how much better off he'll \n be, growing up away from all that until he's ready to take it?''\n Professor McGonagall opened her mouth, changed her mind, \n swallowed, and then said, “Yes — yes, you're right, of course. But \n how is the boy getting here, Dum ble dore?'' She eyed his cloak sud'denly as though she thought he might be hiding Harry underneath \n it.\n “Hagrid's bringing him.''\n “You think it — wise — to trust Hagrid with something as im'portant as this?''\n “I would trust Hagrid with my life,'' said Dum ble dore.\n “I'm not saying his heart isn't in the right place,'' said Professor \n McGonagall grudgingly, “but you can't pretend he's not careless. \n He does tend to — what was that?''\n A low rumbling sound had broken the silence around them. It \n grew steadily louder as they looked up and down the street for \n some sign of a headlight; it swelled to a roar as they both looked up \n at the sky — and a huge motorcycle fell out of the air and landed \n on the road in front of them.\n If the motorcycle was huge, it was nothing to the man sitting \n astride it. He was almost twice as tall as a normal man and at least \n fi ve times as wide. He looked simply too big to be allowed, and so \n wild — long tangles of bushy black hair and beard hid most of his \n face, he had hands the size of trash can lids, and his feet in their \n leather boots were like baby dolphins. In his vast, muscular arms he \n was holding a bundle of blankets.\n “Hagrid,'' said Dum ble dore, sounding relieved. “At last. And \n where did you get that motorcycle?''\n “Borrowed it, Professor Dum ble dore, sir,'' said the giant, climb'ing carefully off the motorcycle as he spoke. “Young Sirius Black \n lent it to me. I've got him, sir.''\n “No problems, were there?''\n “No, sir — house was almost destroyed, but I got him out all \n right before the Muggles started swarmin' around. He fell asleep as \n we was fl yin' over Bristol.''\n Dum ble dore and Professor McGonagall bent forward over the \n bundle of blankets. Inside, just visible, was a baby boy, fast asleep. \n Under a tuft of jet-black hair over his forehead they could see a cu'riously shaped cut, like a bolt of lightning.\n “Is that where — ?'' whispered Professor McGonagall.\n “Yes,'' said Dum ble dore. “He'll have that scar forever.''\n “Couldn't you do something about it, Dum ble dore?''\n “Even if I could, I wouldn't. Scars can come in handy. I have one \n myself above my left knee that is a perfect map of the London Un'derground. Well — give him here, Hagrid — we'd better get this \n over with.''\n Dum ble dore took Harry in his arms and turned toward the \n Dursleys' house.\n “Could I — could I say good-bye to him, sir?'' asked Hagrid. He \n bent his great, shaggy head over Harry and gave him what must \n have been a very scratchy, whiskery kiss. ' en, suddenly, Hagrid \n let out a howl like a wounded dog.\n “Shhh!'' hissed Professor McGonagall, “you'll wake the Mug'gles!''\n “S-s-sorry,'' sobbed Hagrid, taking out a large, spotted handker'chief and burying his face in it. “But I c-c-can't stand it — Lily an' \n James dead — an' poor little Harry off ter live with Muggles —''\n “Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or \n we'll be found,'' Professor McGonagall whispered, patting Hagrid \n gingerly on the arm as Dum ble dore stepped over the low garden \n wall and walked to the front door. He laid Harry gently on the \n doorstep, took a letter out of his cloak, tucked it inside Harry's \n blankets, and then came back to the other two. For a full minute \n the three of them stood and looked at the little bundle; Hagrid's \n shoulders shook, Professor McGonagall blinked furiously, and the \n twinkling light that usually shone from Dum ble dore's eyes seemed \n to have gone out.\n “Well,'' said Dum ble dore fi nally, “that's that. We've no business \n staying here. We may as well go and join the celebrations.''\n “Yeah,'' said Hagrid in a very muffl ed voice, “I'd best get this \n bike away. G'night, Professor McGonagall — Professor Dum'bledore, sir.''\n Wiping his streaming eyes on his jacket sleeve, Hagrid swung \n himself onto the motorcycle and kicked the engine into life; with a \n roar it rose into the air and off into the night.\n “I shall see you soon, I expect, Professor McGonagall,'' said \n Dum ble dore, nodding to her. Professor McGonagall blew her nose \n in reply.\n Dum ble dore turned and walked back down the street. On the \n corner he stopped and took out the silver Put-Outer. He clicked it \n once, and twelve balls of light sped back to their street lamps so \n that Privet Drive glowed suddenly orange and he could make out a \n tabby cat slinking around the corner at the other end of the street. \n He could just see the bundle of blankets on the step of number \n four.\n “Good luck, Harry,'' he murmured. He turned on his heel and \n with a swish of his cloak, he was gone.\n A breeze ruffl ed the neat hedges of Privet Drive, which lay silent \n and tidy under the inky sky, the very last place you would expect \n astonishing things to happen. Harry Potter rolled over inside his \n blankets without waking up. One small hand closed on the letter \n beside him and he slept on, not knowing he was special, not know'ing he was famous, not knowing he would be woken in a few \n hours' time by Mrs. Dursley's scream as she opened the front door \n to put out the milk bottles, nor that he would spend the next few \n weeks being prodded and pinched by his cousin Dudley. . . . He \n couldn't know that at this very moment, people meeting in secret \n all over the country were holding up their glasses and saying in \n hushed voices: “To Harry Potter — the boy who lived!''\n"

In [93]:
corpus = data.lower().split("\n")

In [94]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)

print(len(tokenizer.word_index) + 1)

1276


In [95]:
[corpus[0]]

['mr. and mrs. dursley, of number four, privet drive, were ']

In [96]:
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]

    for i in range(1, len(token_list)):

        n_gram_sequences = token_list[:i+1]
        input_sequences.append(n_gram_sequences)

In [97]:
max_sequence_len = max([len(x) for x in input_sequences])
total_words = len(tokenizer.index_word) + 1

print(max_sequence_len)
print(total_words)

38
1276


In [98]:
input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
# one-hot encoding
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [101]:
tokenizer.word_index["mr."]

KeyError: 'mr.'

In [103]:
# check
print(corpus[0].split())
# print([tokenizer.word_index[word] for word in corpus[0].split()])
print(xs[6])
# print(ys[0])

['mr.', 'and', 'mrs.', 'dursley,', 'of', 'number', 'four,', 'privet', 'drive,', 'were']
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0  25   5  40  16   8 110
 155]


## Training the model

In [104]:
embedding_dim = 64

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(total_words, embedding_dim, input_length = max_sequence_len - 1),
    # Setting a unidirectional LSTM causes some repeatition, Bidirectional causes less
    # tf.keras.layers.LSTM(20),
    
    # Instead, use a bidirectional LSTM
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # One neuron per word
    tf.keras.layers.Dense(total_words, "softmax")
])

model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
optimizer = tf.keras.optimizers.Adam(),
metrics = ["accuracy"])

model.fit(xs, ys, epochs = 500)

Epoch 1/500
142/142 [==============================] - 5s 19ms/step - loss: 6.5104 - accuracy: 0.0334
Epoch 2/500
142/142 [==============================] - 3s 19ms/step - loss: 6.0394 - accuracy: 0.0401
Epoch 3/500
142/142 [==============================] - 3s 18ms/step - loss: 5.9225 - accuracy: 0.0398
Epoch 4/500
142/142 [==============================] - 3s 19ms/step - loss: 5.8361 - accuracy: 0.0390
Epoch 5/500
142/142 [==============================] - 3s 19ms/step - loss: 5.7660 - accuracy: 0.0401
Epoch 6/500
142/142 [==============================] - 3s 19ms/step - loss: 5.7058 - accuracy: 0.0467
Epoch 7/500
142/142 [==============================] - 3s 19ms/step - loss: 5.6509 - accuracy: 0.0509
Epoch 8/500
142/142 [==============================] - 3s 20ms/step - loss: 5.5988 - accuracy: 0.0560
Epoch 9/500
142/142 [==============================] - 3s 19ms/step - loss: 5.5492 - accuracy: 0.0587
Epoch 10/500
142/142 [==============================] - 3s 18ms/step - loss: 5.501

## Prediction

In [108]:
seed_text = "Harry was not a normal boy at all"
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen= max_sequence_len - 1, padding = "pre")
print(token_list)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0  46   7  55   3 192 101  22
   24]]


In [109]:
prediction = model.predict(token_list)
prediction = np.argmax(prediction)
print(tokenizer.index_word[prediction])

1/1 [==============================] - 0s 29ms/step
right


In [112]:
seed_text = "Harry potter is"

for i in range(50):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen= max_sequence_len - 1, padding = "pre")
    prediction = model.predict(token_list)
    prediction = np.argmax(prediction)
    prediction = tokenizer.index_word[prediction]
    seed_text += " " + prediction

print(seed_text)

1/1 [==============================] - 0s 30ms/step
Harry potter is never seen to see the air and landed the letter it black hair and beard hid most of a cat put landed landed landed arrived last wearing beard hid door where he changed of the next lamp too was as or are even still strange dore and at are even


## Enhancements
- larger data sets
- increase embedding dim
- increase # LSTM units (16 ➡️ 150)
- Adam(0.001)